In [8]:
import tensorflow as tf
from tensorflow import keras
import kerastuner as kt
import pandas as pd
from sklearn.model_selection import train_test_split
import Utilidades as ut
import Modelos as md
import numpy as np
from tensorflow.keras.layers import Activation
from sklearn import model_selection

In [9]:
##########################################################

# Redefino función de activación 'relu'

def my_relu(x):
    return tf.keras.backend.relu(x, max_value=1.0)

tf.keras.utils.get_custom_objects().update({'my_relu': Activation(my_relu)})

# Redefino función de activación 'swish'

class Swish(Activation):
    
    def __init__(self, activation, **kwargs):
        super(Swish, self).__init__(activation, **kwargs)
        self.__name__ = 'swish'

def swish(x):
    return (tf.keras.backend.sigmoid(x) * x)

tf.keras.utils.get_custom_objects().update({'swish': Swish(swish)})

##########################################################

In [10]:
# Cargamos el dataframe de training
df_train = pd.read_pickle("Archivos/Neuronales_entrenamiento.pkl")
df_test = pd.read_pickle("Archivos/Neuronales_validacion.pkl")

if ('Opportunity_ID' in df_test):
    df_test = df_test.drop(columns=['Opportunity_ID'])

x_train, y_train = ut.split_labels(df_train)
x_test, y_test = ut.split_labels(df_test)

#Convertimos las fechas a numeros (cantidad de dias transcurridos) y luego las normalizamos
x_train, x_test = ut.conversion_fechas(x_train, x_test)
x_train, x_test = ut.codificar_categoricas(x_train, y_train, x_test, modo='catboost')
x_train, x_test = ut.normalizacion_numericas(x_train, x_test, modo='normalizacion')

x_train = x_train.drop(columns=['Opportunity_Created_Date'])
x_test = x_test.drop(columns=['Opportunity_Created_Date'])

x_train_vector = ut.df_a_vector(x_train)
y_train_vector = ut.df_a_vector(y_train)
x_test_vector = ut.df_a_vector(x_test)
y_test_vector = ut.df_a_vector(y_test)

x_train.shape

(12140, 12)

In [11]:
# Definimos el modelo
def model_builder(hp):
    model = keras.Sequential()
    
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])
    lays = hp.Int('layers', min_value = 4, max_value = 10, step = 1)

    for i in range(lays):
        units_str = 'units_' + f'{i}'
        actv_str = 'activation_func_' + f'{i}'
        model.add(keras.layers.Dense(units = hp.Int(units_str, min_value = 16, max_value = 512, step = 16), 
                                     activation = hp.Choice(actv_str, ['relu', 'swish', 'tanh'])))

    model.add(keras.layers.Dense(units = 1,
                                 activation = 'sigmoid'))

    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = 'binary_crossentropy')

    return model

In [12]:
class CVTuner(kt.engine.tuner.Tuner):
    
    def run_trial(self, trial, x, y, batch_size=32, epochs=1):
        cv = model_selection.TimeSeriesSplit(5)
        val_losses = []
        
        for train_indices, test_indices in cv.split(x):
            x_train, x_test = x[train_indices], x[test_indices]
            y_train, y_test = y[train_indices], y[test_indices]
            model = self.hypermodel.build(trial.hyperparameters)
            model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)
            val_losses.append(model.evaluate(x_test, y_test))
            
        self.oracle.update_trial(trial.trial_id, {'val_loss': np.mean(val_losses)})
        self.save_model(trial.trial_id, model)

In [27]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# # Creamos instancia del tuner
# # Keras Tuner tiene cuatro tuner disponibles: RandomSearch , Hyperband , BayesianOptimization y Sklearn
# tuner = kt.Hyperband(model_builder,
#                      objective = 'val_accuracy', 
#                      max_epochs = 50,
# #                      max_trials = 5,
# #                      factor = 3,
#                      directory = 'Archivo',
#                      project_name = 'Keras_Tuning')

# tuner.search(x_train_vector,
#             y_train_vector,
#             validation_data=(x_test_vector, y_test_vector),
#             epochs = 50,
#             verbose=1,
#             callbacks = [tf.keras.callbacks.EarlyStopping(patience=1)])
    
# models = tuner.get_best_models(num_models=2)
             
# tuner.results_summary()

tuner = CVTuner(hypermodel=model_builder,
                oracle=kt.oracles.Hyperband(
                objective='val_loss',
                max_epochs = 100),
                directory = 'Archivo',
                project_name = 'Keras_Tuning_12_best_new_fts')

# tuner.search(x_train_vector, 
#              y_train_vector, 
#              batch_size=128, 
#              epochs=100)

INFO:tensorflow:Reloading Oracle from existing project Archivo\Keras_Tuning_12_best_new_fts\oracle.json
INFO:tensorflow:Reloading Tuner from Archivo\Keras_Tuning_12_best_new_fts\tuner0.json


In [28]:
tuner.results_summary()